# Camera Trap Processing Thesis

Author: Jameson Andrews\
EDITS:\
09/13/2023, Initial Creation, Jameson

## Load model

In [1]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

preTrainedModel = VGG19()
preTrainedModel.trainable = False
preTrainedModel.summary()


2023-10-29 15:35:52.844440: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-29 15:35:52.844908: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                              

## Create complete Model with Custom output Layer

In [2]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

completeModel = models.Sequential()
completeModel.add(preTrainedModel)
completeModel.add(Dense(100, activation = "relu"))
# completeModel.add(Dense(75, activation = "relu"))
# completeModel.add(Dense(50, activation = "relu"))
# completeModel.add(Dense(25, activation = "relu"))
completeModel.add(Dense(1, activation = "sigmoid"))
completeModel.summary()
completeModel.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 dense (Dense)               (None, 100)               100100    
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 143,767,441
Trainable params: 100,201
Non-trainable params: 143,667,240
_________________________________________________________________


/Users/jamesonandrews/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Orgainze test and training data
Be sure to change file contents

In [3]:
trainingValues = [0] * 859 + [1] * 1209
testValues = [1] * 160 + [0] * 154
trainImages = keras.utils.image_dataset_from_directory( #Takes given directory and automatically organizes it based on subdirectories
    directory='trainingData',
    labels = trainingValues,
    label_mode='binary',
    batch_size=32,
    image_size=(224, 224))
testImages = keras.utils.image_dataset_from_directory(
    directory='testData',
    labels= testValues,
    label_mode='binary',
    batch_size=32,
    image_size=(224, 224))
print (trainImages)

Found 2068 files belonging to 2 classes.
Found 314 files belonging to 2 classes.
<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>


## Prediction before training

In [4]:
testPicture = load_img('../foxNightCrop2.jpg', target_size=(224,224))
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)

2023-10-29 15:35:54.891131: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-10-29 15:35:54.984931: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 286ms/step
[[0.50171673]]


## Train Last two layers

In [5]:
completeModel.fit(trainImages, epochs=10, batch_size=100)

Epoch 1/10


2023-10-29 15:35:55.596511: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


65/65 [==============================] - 21s 315ms/step - loss: 0.6597 - accuracy: 0.5924
Epoch 2/10
65/65 [==============================] - 21s 313ms/step - loss: 0.5640 - accuracy: 0.7510
Epoch 3/10
65/65 [==============================] - 21s 313ms/step - loss: 0.4665 - accuracy: 0.8061
Epoch 4/10
65/65 [==============================] - 21s 314ms/step - loss: 0.4053 - accuracy: 0.8240
Epoch 5/10
65/65 [==============================] - 21s 314ms/step - loss: 0.3687 - accuracy: 0.8477
Epoch 6/10
65/65 [==============================] - 21s 314ms/step - loss: 0.3435 - accuracy: 0.8569
Epoch 7/10
65/65 [==============================] - 21s 314ms/step - loss: 0.3237 - accuracy: 0.8675
Epoch 8/10
65/65 [==============================] - 21s 314ms/step - loss: 0.3086 - accuracy: 0.8791
Epoch 9/10
65/65 [==============================] - 21s 314ms/step - loss: 0.2953 - accuracy: 0.8864
Epoch 10/10
65/65 [==============================] - 21s 314ms/step - loss: 0.2841 - accuracy: 0.8931


## Test Some data

In [6]:
score = completeModel.evaluate(testImages)
print ("loss = ", score[0])
print ("accuracy = ", score[1])

2023-10-29 15:39:21.484976: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 4s 322ms/step - loss: 1.5790 - accuracy: 0.4586
loss =  1.5790038108825684
accuracy =  0.4585987329483032


## See what happens with a deer image

In [7]:
from PIL import Image
testPicture = load_img('testData/testDeer/SEQ86955_IMG_0004.JPG___crop00_md_v5a.0.0.pt.jpg', target_size=(224,224))
# testPicture = load_img('../deerNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../foxNightCrop2.jpg', target_size=(224,224))
# testPicture = load_img('../possumNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../downloadRabbit.jpeg', target_size=(224,224))
# testPicture = load_img('../rabbitNightCrop1.jpg', target_size=(224,224))
testPicture.show()
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)

1/1 [==============================] - 0s 19ms/step
[[0.61766726]]


## Save the model for export

In [8]:
completeModel.save("foxModel.keras")

## Check test data for each animal

In [9]:
# import required module
import os
# assign directory
directory = 'testData/testDeer'
 
# iterate over files in
# that directory
results = [0,0]
# 0 is positives 1 is negatives
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1

print (results)

testData/testDeer/SEQ81957_IMG_0003.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 19ms/step
[[0.8692584]]
testData/testDeer/SEQ80916_IMG_0004.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.29652599]]
testData/testDeer/SEQ86955_IMG_0001.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 19ms/step
[[0.99084693]]
testData/testDeer/SEQ87558_IMG_0003.JPG___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.1083849]]
testData/testDeer/SEQ81957_IMG_0010.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.8413295]]
testData/testDeer/SEQ86931_IMG_0003.JPG___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.35706526]]
testData/testDeer/SEQ81957_IMG_0016.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.8251311]]
testData/testDeer/SEQ87611_IMG_0006.JPG___crop01_md_v5a.0.0.pt.jpg

In [10]:
directory = 'extraTest/testFox'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testFox/SEQ81302_IMG_0003.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.99964094]]
extraTest/testFox/SEQ81302_IMG_0004.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.99925387]]
extraTest/testFox/SEQ81302_IMG_0004.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.999283]]
extraTest/testFox/SEQ81296_IMG_0004.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.8609962]]
extraTest/testFox/SEQ80191_IMG_0010.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step


In [11]:
directory = 'extraTest/testPossum'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testPossum/1980.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.35419402]]
extraTest/testPossum/2005.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9811553]]
extraTest/testPossum/2016.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.6402795]]
extraTest/testPossum/2059.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.020557]]
extraTest/testPossum/1993.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.47812665]]
extraTest/testPossum/2123.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.05959164]]
extraTest/testPossum/1978.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.969159]]
extraTest/testPossum/2130.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.14291257]]
extraTest/test

In [12]:
directory = 'extraTest/testRabbit'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testRabbit/791.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.89565456]]
extraTest/testRabbit/782.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.86427265]]
extraTest/testRabbit/735.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.3999482]]
extraTest/testRabbit/821.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.95744014]]
extraTest/testRabbit/769.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.55054]]
extraTest/testRabbit/832.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9049875]]
extraTest/testRabbit/726.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.18854639]]
extraTest/testRabbit/784.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.5617008]]
extraTest/testRabbit/7

In [13]:
directory = 'extraTest/testSquirrel'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testSquirrel/85.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.01013607]]
extraTest/testSquirrel/83.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.04206301]]
extraTest/testSquirrel/89.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.01678506]]
extraTest/testSquirrel/97.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.7926238]]
extraTest/testSquirrel/91.jpg___crop04_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.00277198]]
extraTest/testSquirrel/96.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.00065388]]
extraTest/testSquirrel/82.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.49110168]]
extraTest/testSquirrel/84.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.01090452]]
extraTest

## Code for data manipulation to creaet more data

In [14]:
# directory = 'trainingData/trainRabbit' #Just change directory name for different animal
 
# # iterate over files in
# # that directory
# results = [0,0,0,0]
# for filename in os.listdir(directory):
#     f = os.path.join(directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         print(f)
#         testPicture = load_img(f, target_size=(224,224))
#         newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
#         newPic.save(f[:-4] + "FlippedImage" + f[-4:])